In [1]:
import requests
import pandas as pd
import os
import sqlite3
from secrets import token

In [2]:
params = {
  'access_key': token,
  'dep_iata': 'SVQ'
}

api_result = requests.get('http://api.aviationstack.com/v1/flights', params)

api_response = api_result.json()

In [3]:
flight_lst = []

for flight in api_response['data'] :
    flight_dict = {}

    flight_dict['flight_date'] = flight['flight_date']
    flight_dict['departure_airport'] = flight['departure']['airport']
    flight_dict['departure_terminal'] = flight['departure']['terminal']
    flight_dict['departure_gate'] = flight['departure']['gate']
    flight_dict['departure_delay'] = flight['departure']['delay']
    flight_dict['departure_scheduled'] = flight['departure']['scheduled']
    flight_dict['departure_estimated'] = flight['departure']['estimated']
    flight_dict['arrival_airport'] = flight['arrival']['airport']
    flight_dict['arrival_terminal'] = flight['arrival']['terminal']
    flight_dict['arrival_gate'] = flight['arrival']['gate']
    flight_dict['arrival_delay'] = flight['arrival']['delay']
    flight_dict['arrival_scheduled'] = flight['arrival']['scheduled']
    flight_dict['arrival_estimated'] = flight['arrival']['estimated']
    flight_dict['airline_name'] = flight['airline']['name']
    flight_dict['flight_number'] = flight['flight']['number']

    flight_lst.append(flight_dict)


flights_df = pd.DataFrame(flight_lst, columns = ['flight_date', 'departure_airport', 'departure_terminal', 'departure_gate', 'departure_delay',
                            'departure_scheduled', 'departure_estimated', 'arrival_airport', 'arrival_terminal', 'arrival_gate', 'arrival_delay',
                            'arrival_scheduled', 'arrival_estimated', 'airline_name', 'flight_number'])

In [4]:
flights_df.head()

,flight_date,departure_airport,departure_terminal,departure_gate,departure_delay,departure_scheduled,departure_estimated,arrival_airport,arrival_terminal,arrival_gate,arrival_delay,arrival_scheduled,arrival_estimated,airline_name,flight_number
0,2022-07-09,San Pablo,1,B17,NaN,2022-07-09T12:35:00+00:00,2022-07-09T12:35:00+00:00,Lanzarote,1,None,NaN,2022-07-09T13:40:00+00:00,2022-07-09T13:40:00+00:00,Iberia,5350
1,2022-07-09,San Pablo,1,B17,NaN,2022-07-09T12:35:00+00:00,2022-07-09T12:35:00+00:00,Lanzarote,1,None,NaN,2022-07-09T13:40:00+00:00,2022-07-09T13:40:00+00:00,Vueling,3160
2,2022-07-09,San Pablo,1,None,39.0,2022-07-09T11:05:00+00:00,2022-07-09T11:05:00+00:00,Gran Canaria,4,None,28.0,2022-07-09T12:20:00+00:00,2022-07-09T12:20:00+00:00,Ryanair,7724
3,2022-07-09,San Pablo,1,None,NaN,2022-07-09T15:15:00+00:00,2022-07-09T15:15:00+00:00,Alicante El Altet,N,None,NaN,2022-07-09T16:20:00+00:00,2022-07-09T16:20:00+00:00,Ryanair,5065
4,2022-07-09,San Pablo,1,None,32.0,2022-07-09T06:45:00+00:00,2022-07-09T06:45:00+00:00,Blagnac,2,A2,20.0,2022-07-09T08:25:00+00:00,2022-07-09T08:25:00+00:00,Ryanair,1192


In [34]:

# if file does not exist write header 
if not os.path.isfile('flights_SVQ.csv'):
       
   flights_df.to_csv('flights_SVQ.csv', header = 'column_names')

else: # else it exists so append without writing the header
   
   flights_df.to_csv('flights_SVQ.csv', mode = 'a', header = False, index = False)

In [36]:
flights_df.columns

Index(['flight_date', 'departure_airport', 'departure_terminal',
       'departure_gate', 'departure_delay', 'departure_scheduled',
       'departure_estimated', 'arrival_airport', 'arrival_terminal',
       'arrival_gate', 'arrival_delay', 'arrival_scheduled',
       'arrival_estimated', 'airline_name', 'flight_number'],
      dtype='object')

In [5]:
flights_df['flight_date'] = pd.to_datetime(flights_df['flight_date'])
flights_df['departure_scheduled'] = pd.to_datetime(flights_df['departure_scheduled'])
flights_df['departure_estimated'] = pd.to_datetime(flights_df['departure_estimated'])
flights_df['arrival_scheduled'] = pd.to_datetime(flights_df['arrival_scheduled'])
flights_df['arrival_estimated'] = pd.to_datetime(flights_df['arrival_estimated'])

In [16]:
conn = sqlite3.connect('flights_db.sqlite3')

c = conn.cursor()

create_flights_query = """
    CREATE TABLE IF NOT EXISTS flight(
        [index] TEXT,
        [flight_date] TEXT,
        [departure_airport] TEXT,
        [departure_terminal] TEXT,
        [departure_gate] TEXT,
        [departure_delay] REAL,
        [departure_scheduled] TEXT,
        [departure_estimated] TEXT,
        [arrival_airport] TEXT,
        [arrival_terminal] TEXT,
        [arrival_gate] TEXT,
        [arrival_delay] REAL,
        [arrival_scheduled] TEXT,
        [arrival_estimated] TEXT,
        [airline_name] TEXT,
        [flight_number] TEXT
    )
"""

c.execute(create_flights_query)
conn.commit()

In [10]:
conn.close()

In [17]:
pd.read_sql_query("SELECT * FROM flight", conn)

,index,flight_date,departure_airport,departure_terminal,departure_gate,departure_delay,departure_scheduled,departure_estimated,arrival_airport,arrival_terminal,arrival_delay,arrival_scheduled,arrival_estimated,airline_name,flight_number


In [19]:
flights_df.to_sql('flight', con = conn, if_exists = 'append')

In [20]:
pd.read_sql_query("SELECT * FROM flight", conn)

,index,flight_date,departure_airport,departure_terminal,departure_gate,departure_delay,departure_scheduled,departure_estimated,arrival_airport,arrival_terminal,arrival_gate,arrival_delay,arrival_scheduled,arrival_estimated,airline_name,flight_number
0,0,2022-07-09 00:00:00,San Pablo,1,B17,NaN,2022-07-09 12:35:00+00:00,2022-07-09 12:35:00+00:00,Lanzarote,1,None,NaN,2022-07-09 13:40:00+00:00,2022-07-09 13:40:00+00:00,Iberia,5350
1,1,2022-07-09 00:00:00,San Pablo,1,B17,NaN,2022-07-09 12:35:00+00:00,2022-07-09 12:35:00+00:00,Lanzarote,1,None,NaN,2022-07-09 13:40:00+00:00,2022-07-09 13:40:00+00:00,Vueling,3160
2,2,2022-07-09 00:00:00,San Pablo,1,None,39.0,2022-07-09 11:05:00+00:00,2022-07-09 11:05:00+00:00,Gran Canaria,4,None,28.0,2022-07-09 12:20:00+00:00,2022-07-09 12:20:00+00:00,Ryanair,7724
3,3,2022-07-09 00:00:00,San Pablo,1,None,NaN,2022-07-09 15:15:00+00:00,2022-07-09 15:15:00+00:00,Alicante El Altet,N,None,NaN,2022-07-09 16:20:00+00:00,2022-07-09 16:20:00+00:00,Ryanair,5065
4,4,2022-07-09 00:00:00,San Pablo,1,None,32.0,2022-07-09 06:45:00+00:00,2022-07-09 06:45:00+00:00,Blagnac,2,A2,20.0,2022-07-09 08:25:00+00:00,2022-07-09 08:25:00+00:00,Ryanair,1192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,2022-07-08 00:00:00,San Pablo,None,B17,9.0,2022-07-08 17:40:00+00:00,2022-07-08 17:40:00+00:00,Charles De Gaulle,3,None,NaN,2022-07-08 20:10:00+00:00,2022-07-08 20:10:00+00:00,Iberia,5908
96,96,2022-07-08 00:00:00,San Pablo,None,A4,30.0,2022-07-08 17:10:00+00:00,2022-07-08 17:10:00+00:00,Charles De Gaulle,2G,None,2.0,2022-07-08 19:40:00+00:00,2022-07-08 19:40:00+00:00,KLM,2076
97,97,2022-07-08 00:00:00,San Pablo,None,None,56.0,2022-07-08 19:45:00+00:00,2022-07-08 19:45:00+00:00,Gatwick,S,None,26.0,2022-07-08 21:30:00+00:00,2022-07-08 21:30:00+00:00,Iberia,7665
98,98,2022-07-08 00:00:00,San Pablo,None,None,NaN,2022-07-08 06:30:00+00:00,2022-07-08 06:30:00+00:00,Lisbon Portela,1,None,NaN,2022-07-08 06:45:00+00:00,2022-07-08 06:45:00+00:00,Air Canada,2677
